In [1]:
import sys
sys.path.append('../')
import wrangle
import explore
import nlp

In [2]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)

df, data_dict = wrangle.wrangle_data()

In [10]:
df = explore.add_target_to_df(df)

targetB = df.targetb

targetB = targetB.reset_index().drop('resp_id',axis=1)

In [11]:
big_df = pd.read_csv('topics.csv', index_col=False)

big_df.drop('Unnamed: 0', axis=1,inplace=True)

In [12]:
big_df['targetB'] = targetB

In [15]:
likely = big_df[big_df.targetB == 1]

In [16]:
likely.shape

(372, 32)

In [17]:
stop_words = ['like', 'plus', 'real', 'love', 'big', 'avoiding', 'mean', 'content', 'people', 'problem', 
              'doing', 'using','research', 'work', 'don', 'make', 'conference', 'yes', 've', 'ha']

stopWords = nlp.set_stop_words(stop_words)

# Respondents Likely To Go

## Sentiment

In [18]:
likely.big_answer.apply(nlp.find_polarity).mean()

0.22151873416100337

In [19]:
likely.big_answer.apply(nlp.find_subjectivity).mean()

0.4278503728167686

## TOTAL KEYWORDS

In [20]:
nlp.show_column_keywords(likely.big_answer, max_df=.8, stop_words=stopWords, ngram_range=(1,3))

['sort',
 'psychology',
 'professional',
 'depending',
 'connection',
 'researchresearch',
 'method',
 'statistic',
 'training',
 'undergraduate',
 'graduate',
 'cost',
 'timing',
 'away',
 'time',
 'scheduling',
 'workshop',
 'small',
 'intimate',
 'international']

## What topics would they be most attracted to at a conference about research?

q21 ideal_topics

In [ ]:
stop_words = ['like', 'plus', 'real', 'love', 'big', 'avoiding', 'mean', 'content', 'people', 'problem', 
              'doing', 'using','research', 'work', 'don', 'make', 'conference', 'good', 'best', 'self', 'report']

In [ ]:
stopWords = nlp.set_stop_words(stop_words)

### keywords

In [21]:
#words
nlp.show_column_keywords(likely.ideal_topics_text, max_df=.5, stop_words=stopWords, ngram_range=(1,3))

['design',
 'innovative',
 'method',
 'communicating',
 'working',
 'nonresearchers',
 'cycle',
 'innovative method',
 'method communicating',
 'participatory',
 'analysis',
 'design participatory',
 'session',
 'new',
 'methodology',
 'case',
 'study',
 'new methodology',
 'case study',
 'participant']

In [22]:
#bi-grams
nlp.show_column_keywords(likely.ideal_topics_text, max_df=.5, stop_words=stopWords, ngram_range=(2,3))

['innovative method',
 'method communicating',
 'design participatory',
 'new methodology',
 'case study',
 'participant recruitment',
 'recruitment strategy',
 'participant recruitment strategy',
 'best practice',
 'space solution',
 'theory practice',
 'sell qualitative',
 'qualitative quantitative',
 'qualitative data',
 'want hear',
 'creative approach',
 'conducting analyzing',
 'cutting edge',
 'application method',
 'data science']

### topics

In [23]:
likely[likely.ideal_topics_text.notnull()].ideal_topics_topic_id.value_counts()

quant/qual/data             77
case_study                  73
new_method, mixed_method    71
ops/ai                      63
Name: ideal_topics_topic_id, dtype: int64

## Who would they expect to see at a conference about research?

q22 Ideal Attendees

In [ ]:
stop_words = ['like', 'plus', 'real', 'love', 'big', 'avoiding', 'mean', 'content', 'people', 'problem', 
              'doing', 'using','research', 'work', 'don', 'make', 'conference']

stopWords = nlp.set_stop_words(stop_words)

In [24]:
#terms
nlp.show_column_keywords(likely.ideal_attendees_text, max_df=.5, stop_words=stopWords)

['company',
 'academic',
 'researcher',
 'public',
 'maker',
 'academic researcher',
 'leading',
 'known',
 'skill',
 'set',
 'expertise',
 'good',
 'sam',
 'ladner',
 'sam ladner',
 'open',
 'woman',
 'minority',
 'walk',
 'just']

In [25]:
#bigrams
nlp.show_column_keywords(likely.ideal_attendees_text, max_df=.5, stop_words=stopWords, ngram_range=(2,3))

['academic researcher',
 'sam ladner',
 'different field',
 'civic tech',
 'steve portigal',
 'working company',
 'kim goodwin',
 'laura klein',
 'way working',
 'industry folk',
 'social scientist',
 'industry thought',
 'thought leader',
 'natalie hanson',
 'industry leader',
 'jared spool',
 'spool steve',
 'google facebook',
 'jared spool steve',
 'spool steve portigal']

In [26]:
likely[likely.ideal_attendees_text.notnull()].ideal_attendees_topic_id.value_counts()

industry, team, product    76
sam ladner, erika hall     55
indi young                 52
experience, jared spool    52
Name: ideal_attendees_topic_id, dtype: int64

## What advice do they have for the Rosenfeld Media team in pursuing a conference?
q23 recommendations

In [27]:
#words
nlp.show_column_keywords(likely.recommendations_text, max_df=.9, stop_words=stopWords, ngram_range=(1,7))

['dont',
 'create',
 'schedule',
 'relevant',
 'event',
 'looking',
 'forward',
 'looking forward',
 'nice',
 'bring',
 'practitioner',
 'researcher',
 'academic',
 'position',
 'similar',
 'tool',
 'strategy',
 'key',
 'opinion',
 'leader']

In [28]:
#ngrams
nlp.show_column_keywords(likely.recommendations_text, max_df=.9, stop_words=stopWords, ngram_range=(2,7))

['looking forward',
 'good luck',
 'event know',
 'ux researcher',
 'advice talk',
 'networking event',
 'introvert extrovert',
 'food good',
 'good mix',
 'mix speaker',
 'way learn',
 'high quality',
 'want attend',
 'healthy food',
 'food option',
 'time break',
 'talk specific',
 'leading team',
 'thought leader',
 'lightning talk']

In [29]:
likely[likely.recommendations_text.notnull()].recommendations_topic_id.value_counts()

speaker, industry      67
group, career, city    57
event, opportunity     52
good, know             45
field, survery         41
Name: recommendations_topic_id, dtype: int64

## Top Documents per Topic

In [30]:
doc_term_matrix, count_vect = nlp.create_wordcount_matrix(likely.recommendations_text, max_df=.3, ngram=(1,3), stop_words=stopWords)

LDA = LatentDirichletAllocation(n_components=4, random_state=42)

LDA.fit(doc_term_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [ ]:
recommendations_dict = {0 : 'speaker, industry', 
                        1 : 'event, opportunity', 
                        2 : 'good, know', 
                        3 : 'field, survery', 
                        4 : 'group, career, city'}

In [31]:
nlp.find_top_documents_per_topic(LDA.transform(doc_term_matrix), likely.recommendations_text, 5)

Top 5 Documents for Topic 0: 

Document 1
I think more events for experienced practitioners would be nice. I’m not sure how’d you’d do that at a conference, but, in general, it’s nice to have space for people who are trying to advance research at their organization. A lot of events seem geared toward getting research off the ground or breaking into the business, which I also love, but sometimes it’s nice to have a conversation that goes beyond the basics. 

Document 2
Short periods of structured interaction are good - NOT "find a buddy and learn about each other" but more like group speed dating: "Create a circle of 5-6 chairs and for 10 minutes your topic is X." 

Provide ways to text Qs to a moderator, who can then ask them of the speaker at the end of the talk - because if I have to ask a Q with a microphone, I will be too distracted by that to concentrate on the talk itself.

Lots of bottled water for breaks - conferences always run out and leave nothing but flavored drinks.




Do